In [0]:
import pandas as pd
import tweepy

In [0]:
api_key = 'Aj8dar7L332QDBvptAp8mVtKz'
secret_key = 'mGjaEJFk8EL4DaBiNp5TN4Zmi5d7MPPCL3g1lwPZQGk8v3d64D'
access_token = '2397096199-0LXa1ze00wodZvqcgAh4Nvro3yKqyyHSWpIEnwI'
access_token_secret = 'T5j56DBAxO3FEYoUex780YEdb4MC5dVnTu9cF9TpLWT3s'

auth = tweepy.OAuthHandler(api_key,secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit = True)

In [0]:
# get tweets based on search query
tweets = pd.DataFrame(columns = ['type','A_handle','A_follower_count','A_following_count','A_listed_count','A_posts',
                                 'A_network_feature_1','A_network_feature_2','A_network_feature_3',
                                 'B_handle','B_follower_count','B_following_count','B_listed_count','B_posts',
                                 'B_network_feature_1','B_network_feature_2','B_network_feature_3'])
tweets_raw = []
text_query = '#chemtrails'
count = 5000

# Pulling individual tweets from query
for tweet in tweepy.Cursor(api.search,
                           q=text_query,
                           count=100, # number of results per page
                           result_type="recent",
                           include_entities=True).items(count): # total number of results
    try:
        tweets_raw.append(tweet)
        # Adding to list that contains all tweets    
        handle = tweet._json['user']['screen_name']
        if tweet.text[:2] == 'RT': # they are retweeting
            rt_handle = tweet._json['retweeted_status']['user']['screen_name']
            rt_followers = tweet._json['retweeted_status']['user']['followers_count']
            rt_following = tweet._json['retweeted_status']['user']['friends_count']
            rt_listed = tweet._json['retweeted_status']['user']['listed_count']
            rt_posts = tweet._json['retweeted_status']['user']['statuses_count']
            content = 'RT'
        elif False:
            # they are quoting a tweet
            pass
        else:
            rt_handle = handle
            content = 'Tweet'
        followers = tweet._json['user']['followers_count']
        following = tweet._json['user']['friends_count']
        listed = tweet._json['user']['listed_count']
        posts = tweet._json['user']['statuses_count']

        tweets = tweets.append(pd.Series({'type':content,'A_handle':handle,'A_follower_count':followers,
                                          'A_following_count':following,'A_listed_count':listed,'A_posts':posts,
                                          'B_handle':rt_handle,'B_follower_count':rt_followers,
                                          'B_following_count':rt_following,'B_listed_count':rt_listed,
                                          'B_posts':rt_posts}), ignore_index = True)
    except:
        continue

tweets.head()

,type,A_handle,A_follower_count,A_following_count,A_listed_count,A_posts,A_network_feature_1,A_network_feature_2,A_network_feature_3,B_handle,B_follower_count,B_following_count,B_listed_count,B_posts,B_network_feature_1,B_network_feature_2,B_network_feature_3
0,RT,Flat67823361,396,383,3,8165,NaN,NaN,NaN,ShowMeTheSky1,406,284,3,4799,NaN,NaN,NaN
1,RT,Cagsil,5237,98,612,430275,NaN,NaN,NaN,Cagsil,5237,98,612,430275,NaN,NaN,NaN
2,RT,WeihlJennifer,414,563,2,42515,NaN,NaN,NaN,BusyDrT,15672,75,191,11738,NaN,NaN,NaN
3,Tweet,BusyDrT,15672,75,191,11738,NaN,NaN,NaN,BusyDrT,15672,75,191,11738,NaN,NaN,NaN
4,RT,cocktail2Funk,614,873,46,21900,NaN,NaN,NaN,prontipronto,513,137,7,5679,NaN,NaN,NaN


Network features 1, 2, and 3 above refer to degree, betweenness and closeness. 

In [0]:
len(tweets)

4622

In [0]:
pd.DataFrame(tweets).to_csv('tweets_'+text_query.strip('#')+'.csv')

In [1]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [2]:
!cp tweets_chemtrails.csv "/content/drive/My Drive/Social Media Analytics/Group Project/Chemtrails"

cp: cannot stat 'tweets_chemtrails.csv': No such file or directory


In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Social Media Analytics/Group Project/Chemtrails/tweets_chemtrails.csv')

In [0]:

import networkx as nx
import numpy as np
import itertools

import matplotlib.pyplot as plt

In [0]:
Graph = nx.from_pandas_edgelist(df,source='A_handle',target='B_handle', edge_attr=None, create_using=nx.DiGraph())

In [9]:
print(nx.info(Graph))

Name: 
Type: DiGraph
Number of nodes: 2050
Number of edges: 2939
Average in degree:   1.4337
Average out degree:   1.4337


In [10]:
#Checking number of nodes
temp = df['A_handle'].append(df['B_handle'])
temp.drop_duplicates()

0         Flat67823361
1               Cagsil
2        WeihlJennifer
3              BusyDrT
4        cocktail2Funk
             ...      
4337      CHEMTRAILSMN
4375    DoraMariamismo
4511           1Naasty
4576        LexAlmight
4593        KindOrgone
Length: 2050, dtype: object

In [0]:
betweenness = nx.betweenness_centrality(Graph)
degree = nx.degree_centrality(Graph)
out_degree = nx.out_degree_centrality(Graph)
in_degree = nx.in_degree_centrality(Graph)
closeness = nx.closeness_centrality(Graph)

In [0]:
bet_df = pd.DataFrame.from_dict(betweenness, orient='index',
                       columns=['Betweenness'])
degree_df = pd.DataFrame.from_dict(degree,orient = 'index', columns = ['Degree'])
close_df = pd.DataFrame.from_dict(closeness, orient = 'index', columns = ['Closeness'])
indegree_df = pd.DataFrame.from_dict(in_degree, orient = 'index', columns = ['In Degree'])
outdegree_df = pd.DataFrame.from_dict(out_degree, orient = 'index', columns = ['Out Degree'])

In [0]:
centrality_metrics = bet_df.merge(degree_df, left_index=True, right_index = True)
centrality_metrics = centrality_metrics.merge(close_df, left_index = True, right_index = True)
centrality_metrics = centrality_metrics.merge(indegree_df, left_index = True, right_index = True)
centrality_metrics = centrality_metrics.merge(outdegree_df, left_index = True, right_index = True)

In [14]:
centrality_metrics.head()

,Betweenness,Degree,Closeness,In Degree,Out Degree
Flat67823361,0.000000,0.001952,0.000000,0.000000,0.001952
ShowMeTheSky1,0.000011,0.028795,0.038931,0.027818,0.000976
Cagsil,0.000000,0.004880,0.003953,0.004392,0.000488
WeihlJennifer,0.000000,0.000488,0.000000,0.000000,0.000488
BusyDrT,0.000000,0.001464,0.000488,0.000976,0.000488


In [0]:
pd.DataFrame(centrality_metrics).to_csv('chemtrails_centrality_metrics.csv')

In [0]:
!cp chemtrails_centrality_metrics.csv "/content/drive/My Drive/Social Media Analytics/Group Project/Chemtrails"